# Assignment 4

In [4]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [11]:
with open("assets/A4_P1_G1",'rb') as f:
    G1 = pickle.load(f)

with open("assets/A4_P1_G2",'rb') as f:
    G2 = pickle.load(f)

with open("assets/A4_P1_G3",'rb') as f:
    G3 = pickle.load(f)

with open("assets/A4_P1_G4",'rb') as f:
    G4 = pickle.load(f)

with open("assets/A4_P1_G5",'rb') as f:
    G5 = pickle.load(f)

In [12]:
# G1 = nx.read_gpickle("assets/A4_P1_G1")
# G2 = nx.read_gpickle("assets/A4_P1_G2")
# G3 = nx.read_gpickle("assets/A4_P1_G3")
# G4 = nx.read_gpickle("assets/A4_P1_G4")
# G5 = nx.read_gpickle("assets/A4_P1_G5")
P1_Graphs = [G1, G2, G3, G4, G5]

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs using any methodology and determine which of the three algorithms generated each graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [93]:
from collections import Counter
def graph_identification():
    answer = []
    for G in P1_Graphs:
        clustering = nx.average_clustering(G)
        shortest_path = nx.average_shortest_path_length(G)
        degrees = [degree for node, degree in G.degree()]
        degree_counts = Counter(degrees)
        if len(degree_counts) > 10:
            answer.append('PA')
        elif shortest_path < 10:
            answer.append('SW_H')
        else:
            answer.append('SW_L')
    return answer


In [ ]:
ans_one = graph_identification()
assert type(ans_one) == list, "You must return a list"


---

## Part 2 - Company Emails

For the second part of this assignment you will be working with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [95]:
G = pickle.load(open('assets/email_prediction_NEW.txt', 'rb'))

print(f"Graph with {len(nx.nodes(G))} nodes and {len(nx.edges(G))} edges")

Graph with 1005 nodes and 16706 edges


In [96]:
list(G.nodes(data=True))[0:10]

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (581, {'Department': 3, 'ManagementSalary': 0.0}),
 (6, {'Department': 25, 'ManagementSalary': 1.0}),
 (65, {'Department': 4, 'ManagementSalary': nan}),
 (64, {'Department': 25, 'ManagementSalary': 1.0}),
 (73, {'Department': 1, 'ManagementSalary': 0.0}),
 (74, {'Department': 1, 'ManagementSalary': 0.0}),
 (459, {'Department': 1, 'ManagementSalary': 0.0}),
 (268, {'Department': 39, 'ManagementSalary': 0.0})]

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features of your choice using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a Pandas series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [226]:
def salary_predictions():
    from sklearn.preprocessing import StandardScaler
    from sklearn.ensemble import RandomForestClassifier

    # Initialize the dataframe, using the nodes as the index
    df = pd.DataFrame(index=G.nodes)
    # Extract node attributes into DF columns
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    # create features for model
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series([degree for node, degree in G.degree()])
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pr'] = pd.Series(nx.pagerank(G))
    df['salary'] = df.apply(
        lambda row: 0
        if np.isnan(row['ManagementSalary']) == True
        else 1,
        axis = 1)
    
    # Define features for model
    features = ['clustering','degree','degree_centrality','closeness','betweeness','pr']
    
    X_train = df[features][df['salary'] == 1]
    y_train = df['ManagementSalary'][df['salary'] == 1]
    X_test = df[features][df['salary'] == 0]


    clf = RandomForestClassifier(random_state = 0)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    clf.fit(X_train_scaled,y_train)

    # test the trained model
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]

    return pd.Series(test_proba, X_test.index)

In [227]:
ans_salary_preds = salary_predictions()
assert type(ans_salary_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_salary_preds) == 252, "The series must be of length 252"


### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [229]:
future_connections = pd.read_csv('assets/Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to:      
1. Create a matrix of features of your choice for the edges found in `future_connections` using Networkx     
2. Train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data     
3. Predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will recieve full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [252]:
def new_connections_predictions():
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.ensemble import RandomForestClassifier

    future_connections['connect'] = future_connections.apply(
        lambda row: 0
        if np.isnan(row['Future Connection']) == True
        else 1,
        axis = 1)

    future_connections['preferential_attachment'] = [list(nx.preferential_attachment(G, [edge]))[0][2] for edge in future_connections.index]
    future_connections['common_neighbors'] = [len(list(nx.common_neighbors(G, edge[0], edge[1]))) for edge in future_connections.index]
    future_connections['jaccard_coefficient'] = [list(nx.jaccard_coefficient(G, [edge]))[0][2] for edge in future_connections.index]
    future_connections['resource_allocation_index'] = [list(nx.resource_allocation_index(G, [edge]))[0][2] for edge in future_connections.index]
    future_connections['adamic_adar_index'] = [list(nx.adamic_adar_index(G, [edge]))[0][2] for edge in future_connections.index]
    future_connections['cn_soundarajan_hopcroft'] = [list(nx.cn_soundarajan_hopcroft(G, ebunch=[edge], community='Department'))[0][2] for edge in future_connections.index]
    future_connections['ra_soundarajan_hopcroft'] =  [list(nx.ra_index_soundarajan_hopcroft(G, ebunch=[edge], community='Department'))[0][2] for edge in future_connections.index]

    features = ['preferential_attachment','common_neighbors','jaccard_coefficient','resource_allocation_index','adamic_adar_index','cn_soundarajan_hopcroft','ra_soundarajan_hopcroft']
    X_train = future_connections[features][future_connections['connect'] == 1]
    y_train = future_connections['Future Connection'][future_connections['connect'] == 1]
    X_test = future_connections[features][future_connections['connect'] == 0]

    clf = RandomForestClassifier(random_state = 0)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    clf.fit(X_train_scaled,y_train)

    # test the trained model
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]

    return pd.Series(test_proba, X_test.index)
    

In [253]:
ans_prob_preds = new_connections_predictions()
assert type(ans_prob_preds) == pd.core.series.Series, "You must return a Pandas series"
assert len(ans_prob_preds) == 122112, "The series must be of length 122112"


In [254]:
ans_prob_preds

(107, 348)    0.000000
(542, 751)    0.012704
(20, 426)     0.470000
(50, 989)     0.015770
(942, 986)    0.011189
                ...   
(165, 923)    0.013465
(673, 755)    0.009808
(939, 940)    0.011189
(555, 905)    0.020304
(75, 101)     0.000000
Length: 122112, dtype: float64